In [13]:
!pip install pytube

In [14]:
import pytube
import cv2
import numpy as np

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [15]:
from tensorflow.keras.models import load_model
model=load_model("/content/gdrive/MyDrive/model3.h5")

In [16]:

img_high=224
img_width=224
url="https://www.youtube.com/watch?v=H7WJcbiTatE"
yt= pytube.YouTube(url)
vid_id=yt.video_id
yt= yt.streams.first()
video=yt.download()

# capturing the video from the given path
cap = cv2.VideoCapture(video)  
total_frames = int(cap.get(7))


#split youtube video to video clips of 2s
clip_len =2
frames=[]
frame_rate = cap.get(5) # frames per second 
vid_len = int(total_frames/frame_rate) # video time  in seconds
intervals = int(vid_len/clip_len) #how many video clips
print(intervals,vid_id)
frame_per_interval = total_frames //intervals #how many frames are in each clips
frame_distance = frame_per_interval//10  # skip some frames 

frame_count = 0   # This variable keeps count of the total frames
clip_count = 1
frame_count_all=0
frame_time=[]

for i in range (total_frames): #iterating over all the frames of the video.
      ret, frame = cap.read()
      if ret ==False:  # Checking if the frame is missing, and if missing, we take the previous frame
          frame = f_prev
      elif ret==True:
          f_prev =frame

      frame_count_all +=1    
      frame_count +=1
    
      if frame_count ==frame_distance:   
          frame_count = 0
          frame=cv2.resize(frame,(img_high,img_width),interpolation=cv2.INTER_AREA)
          frames.append(frame)
          frame_time.append(frame_count_all*(1/frame_rate))
          
        # when we reach to video clip length save the frames
      if i == ((clip_count*frame_per_interval)-1):
          inpt = np.array(frames)
          inpt = inpt.astype('float32')
          inpt -= np.mean(inpt)
          inpt /=np.max(inpt)
          
          frames = []
          p = model.predict(inpt)
          prob = p[:][0][0]

          # find the time of interval
          min_s, sec_s = divmod(frame_time[0], 60)
          start_time=str("%d:%02d" % (min_s, sec_s) ) #Starting time of the interval
          min_e, sec_e = divmod(frame_time[-1],60) 
          end_time= str("%d:%02d" % (min_e, sec_e) )  # Ending time of the interval 
          frame_time=[]

          if prob>=0.50:
              print(prob,start_time,end_time)
              
          clip_count += 1 #next clip
          
          if cv2.waitKey(1) & 0xFF == ord('q'):
              break
cap.release()
cv2.destroyAllWindows()

30 H7WJcbiTatE
0.73235774 0:00 0:02
0.7782864 0:18 0:20
0.67563367 0:20 0:22
0.6522707 0:22 0:24
0.8382882 0:26 0:28
0.884194 0:28 0:30
0.66022956 0:30 0:32
0.72672 0:32 0:34
0.6163566 0:34 0:36
0.5670189 0:36 0:38
0.72977084 0:40 0:42
0.8586043 0:50 0:52
0.77797544 0:52 0:54
0.7407974 0:54 0:56
